# EC Proteomics Analysis - Complete Workflow

This notebook provides a comprehensive analysis of endothelial cell proteomics data, including:
- Data loading and preprocessing
- Principal Component Analysis (PCA) with D2/D5 filtering
- Partial Least Squares Discriminant Analysis (PLS-DA) with 90% threshold
- Visualization and biological interpretation

## Study Overview
This analysis supports the research on endothelial cell metabolic diversity, comparing:
- **HUVEC**: Human umbilical vein endothelial cells
- **iLEC**: Intestinal lymphatic endothelial cells
- **Time points**: Days 2 (proliferation) to day 7 (quiescence)

## 1. Setup and Data Loading

First, let's import the necessary libraries and load our data.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import our custom analysis class
from proteomics_analysis import ProteomicsAnalyzer

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

In [ ]:
# Initialize the analyzer
analyzer = ProteomicsAnalyzer()
print("✅ ProteomicsAnalyzer initialized!")

# Check if data was loaded automatically
if analyzer.data is not None:
    print(f"📊 Data loaded: {analyzer.data.shape[0]} proteins across {analyzer.data.shape[1]} samples")
else:
    print("⚠️ No data loaded automatically. Loading default data file...")
    # Try to load the data manually
    try:
        analyzer.load_data("Suppl_table_1_Px_data.xlsx")
        print(f"📊 Data loaded: {analyzer.data.shape[0]} proteins across {analyzer.data.shape[1]} samples")
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        print("Please ensure 'Suppl_table_1_Px_data.xlsx' is in the current directory")

In [ ]:
# Quick test: Run complete analysis pipeline
print("🚀 Testing complete analysis pipeline...")
try:
    # This will preprocess, normalize, perform PCA and PLS-DA automatically
    results = analyzer.run_analysis(time_points=['D2', 'D5'], plsda_threshold=90.0)
    
    if results:
        print("✅ Complete analysis finished successfully!")
        print(f"📊 Analysis included: {', '.join(results.keys())}")
        
        # Show key results
        if 'plsda' in results and 'significant_proteins' in results['plsda']:
            sig_prots = results['plsda']['significant_proteins']
            bec_count = len(sig_prots.get('BEC_associated', []))
            lec_count = len(sig_prots.get('LEC_associated', []))
            print(f"🔬 Found {bec_count} BEC-associated and {lec_count} LEC-associated proteins (90% threshold)")
        
        print("📁 Results and plots saved to 'results/' directory")
    else:
        print("❌ Analysis failed")
        
except Exception as e:
    print(f"❌ Analysis error: {e}")
    print("💡 You can still run individual analysis steps below")

## 2. Data Exploration

Let's explore the structure of our data and understand the experimental design.

In [ ]:
# Explore data structure
if analyzer.data is not None:
    print("📋 Data Overview:")
    print(f"Total proteins: {analyzer.data.shape[0]}")
    print(f"Total samples: {analyzer.data.shape[1]}")
    print("\n🔬 Sample columns:")
    sample_cols = [col for col in analyzer.data.columns if 'HDBEC' in col or 'iLEC' in col]
    print(f"Found {len(sample_cols)} sample columns")
    print("First 10 samples:", sample_cols[:10])

    # Check for missing values
    missing_percent = (analyzer.data[sample_cols].isnull().sum() / len(analyzer.data)) * 100
    print(f"\n📊 Missing values: {missing_percent.mean():.1f}% average per sample")
else:
    print("❌ No data available. Please load data first.")

In [ ]:
# Create sample metadata
metadata = []
for col in sample_cols:
    parts = col.split('_')
    if len(parts) >= 3:
        cell_type = parts[0]
        timepoint = parts[1]
        replicate = parts[2]
        metadata.append({
            'sample': col,
            'cell_type': cell_type,
            'timepoint': timepoint,
            'replicate': replicate,
            'group': f"{cell_type}_{timepoint}"
        })

metadata_df = pd.DataFrame(metadata)
print("📊 Sample Metadata:")
print(metadata_df.groupby(['cell_type', 'timepoint']).size().unstack(fill_value=0))

## 3. Principal Component Analysis (PCA)

We'll perform PCA to understand the main sources of variation in our data, with special focus on D2 and D5 time points as requested.

In [ ]:
# Perform PCA for all data first
print("🔬 Performing PCA on all samples...")
try:
    pca_all = analyzer.perform_pca(n_components=10)
    
    if pca_all:
        print("✅ PCA completed successfully!")
        print(f"📊 PC1 explains {pca_all['explained_variance_ratio'][0]:.2%} of variance")
        print(f"📊 PC2 explains {pca_all['explained_variance_ratio'][1]:.2%} of variance")
        print(f"📊 First 2 PCs explain {sum(pca_all['explained_variance_ratio'][:2]):.2%} of total variance")
    else:
        print("❌ PCA failed - check data preprocessing")
except Exception as e:
    print(f"❌ PCA error: {e}")
    print("💡 This might be due to data preprocessing issues")

In [ ]:
# Now perform PCA specifically for D2 and D5 samples as requested
print("🔬 Performing PCA for D2 and D5 samples only...")
try:
    pca_d2_d5 = analyzer.perform_pca(time_points=['D2', 'D5'], n_components=5)

    if pca_d2_d5:
        print("✅ D2/D5 PCA completed successfully!")
        print(f"📊 PC1 explains {pca_d2_d5['explained_variance_ratio'][0]:.2%} of variance")
        print(f"📊 PC2 explains {pca_d2_d5['explained_variance_ratio'][1]:.2%} of variance")
    else:
        print("❌ D2/D5 PCA failed - check data filtering")
except Exception as e:
    print(f"❌ D2/D5 PCA error: {e}")
    print("💡 Check that D2 and D5 samples exist in your data")

## 4. Partial Least Squares Discriminant Analysis (PLS-DA)

We'll perform PLS-DA to identify proteins that discriminate between BEC and LEC cell types, using a 90% threshold for protein selection.

In [ ]:
# Perform PLS-DA with 90% threshold as requested
print("🔬 Performing PLS-DA with 90% threshold...")
plsda_results = analyzer.perform_plsda(threshold_percentile=0.9, n_permutations=100)

if plsda_results:
    print("✅ PLS-DA completed successfully!")
    
    # Display key results
    if 'significant_proteins' in plsda_results:
        bec_proteins = plsda_results['significant_proteins'].get('BEC_associated', [])
        lec_proteins = plsda_results['significant_proteins'].get('LEC_associated', [])
        
        print(f"📊 Found {len(bec_proteins)} BEC-associated proteins")
        print(f"📊 Found {len(lec_proteins)} LEC-associated proteins")
        
        if len(bec_proteins) > 0:
            print(f"\n🔝 Top 5 BEC proteins: {bec_proteins[:5]}")
        if len(lec_proteins) > 0:
            print(f"🔝 Top 5 LEC proteins: {lec_proteins[:5]}")
    
    # Display model performance
    if 'model_performance' in plsda_results:
        perf = plsda_results['model_performance']
        print(f"\n📈 Model Performance:")
        print(f"   R² Score: {perf.get('r2_score', 'N/A')}")
        print(f"   Cross-validation Score: {perf.get('cv_score', 'N/A')}")
else:
    print("❌ PLS-DA failed - check data preprocessing")

## 5. Visualization and Results

Let's create comprehensive visualizations of our results.

In [ ]:
# Create a summary visualization
if analyzer.data is not None and 'sample_cols' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('EC Proteomics Analysis Summary', fontsize=16, fontweight='bold')

    # Plot 1: PCA Explained Variance
    if pca_all and 'explained_variance_ratio' in pca_all:
        explained_var = pca_all['explained_variance_ratio'][:5]
        axes[0,0].bar(range(1, len(explained_var)+1), explained_var * 100)
        axes[0,0].set_xlabel('Principal Component')
        axes[0,0].set_ylabel('Explained Variance (%)')
        axes[0,0].set_title('PCA Explained Variance')
        axes[0,0].grid(True, alpha=0.3)

    # Plot 2: Sample counts by cell type and timepoint
    if 'metadata_df' in locals():
        sample_counts = metadata_df.groupby(['cell_type', 'timepoint']).size().unstack(fill_value=0)
        sample_counts.plot(kind='bar', ax=axes[0,1], stacked=True)
        axes[0,1].set_title('Sample Distribution')
        axes[0,1].set_xlabel('Cell Type')
        axes[0,1].set_ylabel('Number of Samples')
        axes[0,1].tick_params(axis='x', rotation=45)
    else:
        axes[0,1].text(0.5, 0.5, 'Metadata not available', ha='center', va='center', transform=axes[0,1].transAxes)
        axes[0,1].set_title('Sample Distribution')

    # Plot 3: Missing data heatmap
    missing_data = analyzer.data[sample_cols].isnull().sum().values.reshape(1, -1)
    im = axes[1,0].imshow(missing_data, cmap='Reds', aspect='auto')
    axes[1,0].set_title('Missing Data per Sample')
    axes[1,0].set_xlabel('Samples')
    axes[1,0].set_yticks([])
    plt.colorbar(im, ax=axes[1,0], label='Missing Values')

    # Plot 4: Protein significance counts
    if 'plsda_results' in locals() and plsda_results and 'significant_proteins' in plsda_results:
        sig_prots = plsda_results['significant_proteins']
        counts = [len(sig_prots.get('BEC_associated', [])), 
                  len(sig_prots.get('LEC_associated', []))]
        axes[1,1].bar(['BEC-associated', 'LEC-associated'], counts, 
                      color=['lightcoral', 'lightblue'])
        axes[1,1].set_title('Significant Proteins (90% threshold)')
        axes[1,1].set_ylabel('Number of Proteins')
        
        # Add count labels on bars
        for i, count in enumerate(counts):
            axes[1,1].text(i, count + 10, str(count), ha='center', fontweight='bold')
    else:
        axes[1,1].text(0.5, 0.5, 'PLS-DA results not available', 
                       ha='center', va='center', transform=axes[1,1].transAxes)
        axes[1,1].set_title('PLS-DA Results')

    plt.tight_layout()
    plt.show()
else:
    print("❌ No data available for visualization. Please load data first.")

## 6. Interactive Visualizations

Let's create interactive plots for better exploration of the results.

In [ ]:
# Create interactive PCA plot for D2/D5 data if available
if pca_d2_d5 and 'transformed_data' in pca_d2_d5:
    pca_data = pca_d2_d5['transformed_data']
    pca_samples = pca_d2_d5['sample_names']
    
    # Create metadata for PCA samples
    pca_metadata = []
    for sample in pca_samples:
        parts = sample.split('_')
        if len(parts) >= 3:
            pca_metadata.append({
                'sample': sample,
                'cell_type': parts[0],
                'timepoint': parts[1],
                'replicate': parts[2]
            })
    
    pca_meta_df = pd.DataFrame(pca_metadata)
    
    # Create interactive PCA plot
    fig = px.scatter(
        x=pca_data[:, 0],
        y=pca_data[:, 1],
        color=pca_meta_df['cell_type'],
        symbol=pca_meta_df['timepoint'],
        hover_data={'sample': pca_meta_df['sample']},
        title='PCA Analysis - D2 and D5 Samples Only',
        labels={
            'x': f'PC1 ({pca_d2_d5["explained_variance_ratio"][0]:.2%} variance)',
            'y': f'PC2 ({pca_d2_d5["explained_variance_ratio"][1]:.2%} variance)',
            'color': 'Cell Type',
            'symbol': 'Timepoint'
        }
    )
    fig.update_layout(width=800, height=600)
    fig.show()
else:
    print("⚠️ PCA D2/D5 data not available for plotting")

## 7. Export Results

Let's export our significant findings for further analysis.

In [ ]:
# Export significant proteins to CSV
if plsda_results and 'significant_proteins' in plsda_results:
    sig_prots = plsda_results['significant_proteins']
    
    # Create results directory if it doesn't exist
    import os
    os.makedirs('results', exist_ok=True)
    
    # Export BEC-associated proteins
    if 'BEC_associated' in sig_prots and len(sig_prots['BEC_associated']) > 0:
        bec_df = pd.DataFrame({
            'protein': sig_prots['BEC_associated']
        })
        bec_df.to_csv('results/BEC_associated_proteins_90pct.csv', index=False)
        print(f"✅ Exported {len(bec_df)} BEC-associated proteins")
    
    # Export LEC-associated proteins
    if 'LEC_associated' in sig_prots and len(sig_prots['LEC_associated']) > 0:
        lec_df = pd.DataFrame({
            'protein': sig_prots['LEC_associated']
        })
        lec_df.to_csv('results/LEC_associated_proteins_90pct.csv', index=False)
        print(f"✅ Exported {len(lec_df)} LEC-associated proteins")
    
    print("\n📁 Results saved to 'results/' directory")
else:
    print("⚠️ No significant proteins found to export")

## 8. Biological Interpretation

Let's examine some of the key findings and their biological significance.

In [ ]:
# Summary of analysis results
print("🧬 BIOLOGICAL INTERPRETATION")
print("=" * 50)

if plsda_results and 'significant_proteins' in plsda_results:
    sig_prots = plsda_results['significant_proteins']
    
    bec_count = len(sig_prots.get('BEC_associated', []))
    lec_count = len(sig_prots.get('LEC_associated', []))
    
    print(f"📊 SUMMARY:")
    print(f"   • {bec_count} proteins significantly associated with BEC phenotype")
    print(f"   • {lec_count} proteins significantly associated with LEC phenotype")
    print(f"   • 90% threshold applied for high-confidence protein selection")
    
    if pca_d2_d5:
        var_explained = sum(pca_d2_d5['explained_variance_ratio'][:2]) * 100
        print(f"   • First 2 PCs explain {var_explained:.1f}% variance in D2/D5 samples")
    
    print(f"\n🔬 BIOLOGICAL SIGNIFICANCE:")
    print(f"   • BEC vs LEC differences reflect distinct endothelial cell fates")
    print(f"   • D2 vs D5 comparison captures early differentiation dynamics")
    print(f"   • Identified proteins may serve as biomarkers or therapeutic targets")
    
    print(f"\n📈 NEXT STEPS:")
    print(f"   • Pathway enrichment analysis of significant proteins")
    print(f"   • Validation in independent datasets")
    print(f"   • Functional studies of key candidate proteins")
    
else:
    print("⚠️ Analysis results not available for interpretation")

print("\n✅ Analysis completed successfully!")
print("📁 Check the 'results/' directory for exported files")

## 9. Additional Analysis Options

Here are some additional analyses you can perform with the `ProteomicsAnalyzer`:

In [ ]:
# Example of additional analyses you can run
print("🔧 ADDITIONAL ANALYSIS OPTIONS:")
print("=" * 40)
print("\n1. Full comprehensive analysis:")
print("   analyzer.run_analysis()")

print("\n2. Custom PLS-DA with different thresholds:")
print("   analyzer.perform_plsda(threshold=0.95)  # 95% threshold")
print("   analyzer.perform_plsda(threshold=0.85)  # 85% threshold")

print("\n3. PCA with specific time points:")
print("   analyzer.perform_pca(time_points=['D3', 'D4'])")
print("   analyzer.perform_pca(time_points=['D6', 'D7'])")

print("\n4. Subset analysis by cell type:")
print("   # This would require modifying the analyzer for cell-type specific analysis")

print("\n5. Time-course analysis:")
print("   # Analyze protein changes across all time points")

print("\n📖 For more details, see the ProteomicsAnalyzer class documentation")